In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'store_operation',
    proc_date,
    'temp_rop_store_operation'
)

In [0]:
spark.sql(f""" CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_rop.store_operation LIKE {catalog_name}.udp_wcm_bronze_rop.store_operation""")

In [0]:
spark.sql(f"""ALTER TABLE {catalog_name}.udp_wcm_silver_rop.store_operation
SET TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_rop.store_operation
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
        INSERT INTO {catalog_name}.udp_wcm_silver_rop.store_operation 
        SELECT * 
        FROM temp_rop_store_operation
        WHERE proc_date = DATE('{proc_date}')
    """)